   # Hello PyTorch with MLflow

In this example, we like to demonstrate that the example code used in hello-pt-tb with PyTorch Tensorboard tracking can be simply switch to using MLflow as experimental tracking without changing the code



Example of using [NVIDIA FLARE](https://nvflare.readthedocs.io/en/main/index.html) to train an image classifier using federated averaging ([FedAvg]([FedAvg](https://arxiv.org/abs/1602.05629))) and [PyTorch](https://pytorch.org/) as the deep learning training framework. This example also highlights the streaming capability from the clients to the server with Tensorboard SummaryWriter sender syntax, but with a MLFlow receiver

> **_NOTE:_** This example uses the [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset and will load its data within the trainer code.


### 1. Install NVIDIA FLARE

Follow the [Installation](https://nvflare.readthedocs.io/en/main/getting_started.html#installation) instructions.
Install additional requirements:


In [ ]:
%%bash
pip3 install torch torchvision tensorboard mlflow

### 2. Change Configuration

in fed_server_config.json

add the following to the components
```
{
      "id": "mlflow_receiver",
      "path": "nvflare.app_opt.tracking.mlflow.mlflow_receiver.MLFlowReceiver",
      "args": {
        "kwargs": {"experiment_name": "hello-pt-experiments"},
        "artifact_location": "artifacts"
      }
}
```
This indicate that we are register the MLFLow Receiver in additional to the Tensorboard Receiver 




### 3. Run the experiment

Use nvflare simulator to run the hello-examples:

```
nvflare simulator -w /tmp/nvflare/ -n 2 -t 2 examples/hello-pt-tb-mlflow/hello-pt-tb-mlflow-job1
```

In [2]:
%%bash
cd $NVFLARE_HOME

nvflare simulator -w /tmp/nvflare/ -n 2 -t 2 examples/hello-pt-tb-mlflow/hello-pt-tb-mlflow-job1

Traceback (most recent call last):
  File "/workspace/venvs/NVFlare/519nvflare-env/bin/nvflare", line 5, in <module>
    from nvflare.cli import main
  File "/workspace/repos/NVFlare_experiment2/nvflare/cli.py", line 20, in <module>
    from nvflare.dashboard.cli import define_dashboard_parser, handle_dashboard
  File "/workspace/repos/NVFlare_experiment2/nvflare/dashboard/cli.py", line 24, in <module>
    from nvflare.dashboard.application.blob import _write
  File "/workspace/repos/NVFlare_experiment2/nvflare/dashboard/application/__init__.py", line 19, in <module>
    from flask_sqlalchemy import SQLAlchemy
  File "/workspace/venvs/NVFlare/519nvflare-env/lib/python3.10/site-packages/flask_sqlalchemy/__init__.py", line 16, in <module>
    from sqlalchemy import event, inspect, orm
  File "/workspace/venvs/NVFlare/519nvflare-env/lib/python3.10/site-packages/sqlalchemy/orm/__init__.py", line 309, in <module>
    from .events import AttributeEvents
  File "/workspace/venvs/NVFlare/519nv

Process is interrupted.


### 4. Tensorboard Tracking

On the client side, we are still using the TensorBoard SummaryWriter as the `AnalyticsSender`. 

Instead of writing to TB files, it actually generates NVFLARE events of type `analytix_log_stats`.
The `ConvertToFedEvent` widget will turn the event `analytix_log_stats` into a fed event `fed.analytix_log_stats`,
which will be delivered to the server side.

On the server side, the `TBAnalyticsReceiver` is configured to process `fed.analytix_log_stats` events,
which writes received TB data into appropriate TB files on the server.

To view training metrics that are being streamed to the server, run:

```
tensorboard --logdir=/tmp/nvflare/simulate_job/tb_events
```

In [ ]:
%%bash
tensorboard --logdir=/tmp/nvflare/simulate_job/tb_events

### 5. MLFlow tracking

On the server side, we also configured `MLFlowReceiver` to process `fed.analytix_log_stats` events,
which writes received and write to the MLFlow backendstore

To view training metrics that are being streamed to the server, run:

```
mlflow ui --backend-store-uri=/tmp/nvflare/mlruns
```

Then 

Look at the URL in browser http://localhost:5000/